# Raw Data Cleaning
We set up a new Jupyter Notebook to backup the data as `rawdata` to avoid bugs and mistakes happening.

In [1]:
import pandas as pd

In [2]:
rawdata = pd.read_csv('merged_dataframe.csv')
rawdata = rawdata.sort_values(by='Year', ascending=True)
rawdata['Entity'] = 'China'
rawdata['Code'] = 'CHN'

cols = rawdata.columns.tolist()
col_to_drop = cols[20] 
rawdata = rawdata.drop(columns=[col_to_drop])

cols = rawdata.columns.tolist()
cols.insert(3, cols.pop(6))
cols.insert(4, cols.pop(7))

rawdata = rawdata[cols]
rawdata = rawdata.reset_index(drop=True)


Consistent Scale

In [3]:
columns_to_convert = [
    'Foreign Direct Investment', 
    'Unemployment rate', 
    'Government health expenditure', 
    'Government revenues', 
    'Age dependency ratio', 
    'Research and development spending'
]
for column in columns_to_convert:
    if column in rawdata.columns:
        rawdata[column] = rawdata[column].apply(lambda x: x * 100 if pd.notnull(x) else x)

rawdata.to_csv('Raw_data.csv', index=False)

rawdata

,Year,Entity,Code,GDP per Capita,Population,"Government expenditure on education, total (% of GDP)",Government expenditure on primary education as % of GDP (%),Annual Working Hours,Consumer Price Index,Government Expenditure,...,Government health expenditure,Life Expectancy,Government revenues,Age dependency ratio,Research and development spending,Taxes on goods and services (as a share of GDP),"Taxes on income, profits and capital gains (TIPCG) (as a share of GDP)",Human Development Index,Gender Development Index,Productivity
0,1970,China,CHN,1292.5070,8.225345e+08,NaN,NaN,1976.3120,NaN,NaN,...,NaN,NaN,NaN,80.132945,NaN,NaN,NaN,NaN,NaN,1. 4994909
1,1971,China,CHN,1382.5481,8.432854e+08,1.504148,NaN,1975.7937,NaN,NaN,...,NaN,57.472110,NaN,79.802335,NaN,NaN,NaN,NaN,NaN,1.5798508
2,1972,China,CHN,1310.6804,8.628404e+08,1.620546,NaN,1975.5077,NaN,NaN,...,NaN,58.388600,NaN,78.953924,NaN,NaN,NaN,NaN,NaN,1. 492279
3,1973,China,CHN,1396.8113,8.816521e+08,1.697113,NaN,1975.3647,NaN,NaN,...,NaN,59.367887,NaN,78.661213,NaN,NaN,NaN,NaN,NaN,1. 5905421
4,1974,China,CHN,1392.1698,8.993677e+08,1.754547,NaN,1975.1897,NaN,NaN,...,NaN,60.144828,NaN,78.936067,NaN,NaN,NaN,NaN,NaN,1. 5722494
5,1975,China,CHN,1428.9760,9.151247e+08,1.740779,NaN,1974.8975,NaN,NaN,...,NaN,60.702793,NaN,79.252391,NaN,NaN,NaN,NaN,NaN,1. 5965048
6,1976,China,CHN,1350.1012,9.293759e+08,1.779506,NaN,1974.2075,NaN,NaN,...,NaN,61.613119,NaN,79.236954,NaN,NaN,NaN,NaN,NaN,1. 4927917
7,1977,China,CHN,1424.0562,9.425813e+08,1.807332,NaN,1973.4353,NaN,NaN,...,NaN,62.365282,NaN,77.057473,NaN,NaN,NaN,NaN,NaN,1. 5616083
8,1978,China,CHN,1618.6330,9.551389e+08,2.022968,NaN,1972.7274,NaN,NaN,...,NaN,62.818361,NaN,73.129774,NaN,NaN,NaN,NaN,NaN,1. 7562629
9,1979,China,CHN,1677.2329,9.682989e+08,2.015320,NaN,1972.1038,NaN,NaN,...,NaN,63.483663,NaN,70.219607,NaN,NaN,NaN,NaN,NaN,1. 7937474


# Data Quality KPIs Test
After data collection, we get the raw data, which still needs to be cleaned. To test data quality, we use several KPIs to develop the data quality.


## Test DataFrame

In [4]:
import requests

def fetch_indicator(country_code, indicator, date_range):
    # Build the URL for the API request
    url = f'https://api.worldbank.org/v2/country/{country_code}/indicator/{indicator}?date={date_range}&format=json&per_page=500'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if data and len(data) > 1:
            return pd.DataFrame([
                {
                    'Year': item['date'],
                    indicator: item['value']
                }
                for item in data[1]
            ])
        else:
            print(f"No data found for {indicator}.")
            return pd.DataFrame()
    else:
        print(f"Failed to retrieve data for {indicator}.")
        return pd.DataFrame()


In [5]:
country_code = 'CN'
date_range = '1970:2021'
indicators = {
    'NY.GDP.PCAP.CD': 'GDP per Capita',
    'SP.POP.TOTL': 'Population',
    'SE.XPD.TOTL.GD.ZS': 'Government expenditure on education, total (% of GDP)',
    'UIS.XGDP.1.FSGOV': 'Government expenditure on primary education as % of GDP (%)',
    'FP.CPI.TOTL': 'Consumer Price Index',
    'GB.XPD.RSDV.GD.ZS': 'Research And Development Expenditure (% Of GDP)'
}

In [6]:
frames = [fetch_indicator(country_code, indicator, date_range).rename(columns={indicator: name})
          for indicator, name in indicators.items()]

testdata = pd.concat(frames, axis=1)
testdata = testdata.loc[:,~testdata.columns.duplicated()]  # Remove duplicate columns, if any
testdata = testdata.sort_values(by='Year', ascending=True)  # Sort by Year

testdata.dropna(how='all', subset=indicators.values(), inplace=True)

testdata = testdata.reset_index(drop=True)
# testdata.to_csv('Test_data.csv', index=False)


In [7]:
extra_test_new = pd.read_csv('extra_test.csv')
testdata_new = testdata
extra_test_new['Year'] = extra_test_new['Year'].astype('int64')
testdata_new['Year'] = testdata_new['Year'].astype('int64')
merged_data = pd.merge(testdata_new, extra_test_new, on='Year', how='outer', suffixes=('', '_extra'))

for col in extra_test_new.columns:
    if '_extra' in col:
        original_col_name = col.replace('_extra', '')
        merged_data[original_col_name] = merged_data.apply(lambda row: row[col] if pd.notna(row[col]) else row[original_col_name], axis=1)

merged_data_final = merged_data.drop(columns=[col for col in merged_data.columns if '_extra' in col])
testdata = merged_data_final
testdata.to_csv('Test_data.csv', index=False)
testdata

,Year,GDP per Capita,Population,"Government expenditure on education, total (% of GDP)",Government expenditure on primary education as % of GDP (%),Consumer Price Index,Research And Development Expenditure (% Of GDP),Entity,Code,Annual Working Hours,Foreign Direct Investment,Unemployment rate,Government health expenditure,Life Expectancy,Government revenues,Age dependency ratio,Human Development Index,Gender Development Index
0,1970,113.162578,818315000,NaN,NaN,NaN,NaN,China,CHN,1976.3120,NaN,NaN,NaN,56.6070,NaN,80.52,NaN,NaN
1,1971,118.654144,841105000,1.388740,NaN,NaN,NaN,China,CHN,1975.7937,NaN,NaN,NaN,57.5653,NaN,80.19,NaN,NaN
2,1972,131.885559,862030000,1.604760,NaN,NaN,NaN,China,CHN,1975.5077,NaN,NaN,NaN,58.4184,NaN,79.48,NaN,NaN
3,1973,157.089111,881940000,1.650820,NaN,NaN,NaN,China,CHN,1975.3647,NaN,NaN,NaN,59.3891,NaN,78.83,NaN,NaN
4,1974,160.147688,900350000,1.761150,NaN,NaN,NaN,China,CHN,1975.1897,NaN,NaN,NaN,60.1683,NaN,79.06,NaN,NaN
5,1975,178.339614,916395000,1.708500,NaN,NaN,NaN,China,CHN,1974.8975,NaN,NaN,NaN,60.9170,NaN,79.51,NaN,NaN
6,1976,165.404262,930685000,1.816570,NaN,NaN,NaN,China,CHN,1974.2075,NaN,NaN,NaN,61.7956,NaN,79.76,NaN,NaN
7,1977,185.420537,943455000,1.749540,NaN,NaN,NaN,China,CHN,1973.4353,NaN,NaN,NaN,62.5379,NaN,78.43,NaN,NaN
8,1978,228.519313,956165000,1.958840,NaN,NaN,NaN,China,CHN,1972.7274,NaN,NaN,NaN,63.2177,NaN,74.77,NaN,NaN
9,1979,272.146923,969005000,2.148050,NaN,NaN,NaN,China,CHN,1972.1038,NaN,NaN,NaN,63.8649,NaN,71.06,NaN,NaN


## Consistency
Some data like `GDP per Capita` and `Population`... had the official data to compute matched data.

In [8]:
def calculate_consistency(rawdata, testdata):
    consistency_results = {column: 100 for column in rawdata.columns}
    for column in rawdata.columns.intersection(testdata.columns):
        merged_data = pd.merge(rawdata[[column]], testdata[[column]], left_index=True, right_index=True, how='outer', suffixes=('_raw', '_test'))
        matches = merged_data.apply(lambda x: x[column + '_raw'] == x[column + '_test'] if pd.notna(x[column + '_raw']) or pd.notna(x[column + '_test']) else True, axis=1)
        consistency_percentage = (matches.sum() / len(merged_data)) * 100
        consistency_results[column] = consistency_percentage

    consistency_df = pd.DataFrame(list(consistency_results.items()), columns=['Column', 'Consistency'])
    average_consistency = consistency_df['Consistency'].sum() / len(consistency_results)
    consistency_df['Consistency'] = consistency_df['Consistency'].apply(lambda x: f"{x:.2f}%")
    return consistency_df, average_consistency

consistency_df, average_consistency = calculate_consistency(rawdata, testdata)

print(consistency_df)
print(f"Average Consistency: {average_consistency:.2f}%")

                                               Column Consistency
0                                                Year     100.00%
1                                              Entity     100.00%
2                                                Code     100.00%
3                                      GDP per Capita       0.00%
4                                          Population       0.00%
5   Government expenditure on education, total (% ...       1.92%
6   Government expenditure on primary education as...     100.00%
7                                Annual Working Hours     100.00%
8                                Consumer Price Index      30.77%
9                              Government Expenditure     100.00%
10                          Foreign Direct Investment      21.15%
11                                  Unemployment rate      40.38%
12                      Government health expenditure      61.54%
13                                    Life Expectancy       0.00%
14        

## Reliability

In [9]:
def calculate_reliability(rawdata, testdata):
    reliability_results = {column: 100 for column in rawdata.columns}
    for column in rawdata.columns.intersection(testdata.columns):
        merged_data = pd.merge(rawdata[[column]], testdata[[column]], left_index=True, right_index=True, how='outer', suffixes=('_raw', '_test'))
        merged_data[column + '_raw'] = pd.to_numeric(merged_data[column + '_raw'], errors='coerce')
        merged_data[column + '_test'] = pd.to_numeric(merged_data[column + '_test'], errors='coerce')
        percentage_differences = merged_data.apply(lambda x: abs(x[column + '_raw'] - x[column + '_test']) / x[column + '_test'] * 100 if pd.notna(x[column + '_raw']) and pd.notna(x[column + '_test']) else None, axis=1)
        reliability_percentage = 100 - percentage_differences.mean()
        reliability_results[column] = reliability_percentage if pd.notna(reliability_percentage) else 100

    reliability_df = pd.DataFrame(list(reliability_results.items()), columns=['Column', 'Reliability'])
    average_reliability = reliability_df['Reliability'].mean()
    reliability_df['Reliability'] = reliability_df['Reliability'].apply(lambda x: f"{x:.2f}%" if pd.notna(x) else 'NA')
    return reliability_df, average_reliability

reliability_df, average_reliability = calculate_reliability(rawdata, testdata)

# Print the results
print(reliability_df)
print(f"Average Reliability: {average_reliability:.2f}%")

                                               Column Reliability
0                                                Year     100.00%
1                                              Entity     100.00%
2                                                Code     100.00%
3                                      GDP per Capita    -331.51%
4                                          Population      99.36%
5   Government expenditure on education, total (% ...      97.49%
6   Government expenditure on primary education as...     100.00%
7                                Annual Working Hours     100.00%
8                                Consumer Price Index      97.82%
9                              Government Expenditure     100.00%
10                          Foreign Direct Investment      77.50%
11                                  Unemployment rate      96.98%
12                      Government health expenditure      96.51%
13                                    Life Expectancy      98.94%
14        

## Completeness

In [10]:
def calculate_completeness(data):
    completeness_per_column = {}
    for column in data.columns:
        non_na_count = data[column].notna().sum()
        total_count = data[column].shape[0]
        completeness = (non_na_count / total_count) * 100
        completeness_per_column[column] = completeness

    completeness_df = pd.DataFrame(list(completeness_per_column.items()), columns=['Column', 'Completeness (%)'])
    
    total_non_na_count = data.notna().sum().sum()
    total_values = data.shape[0] * data.shape[1]
    overall_completeness = (total_non_na_count / total_values) * 100

    return completeness_df, overall_completeness
completeness_df, overall_completeness = calculate_completeness(rawdata)

# Print the results
print(completeness_df)
print(f"Overall Completeness: {overall_completeness:.2f}%")

                                               Column  Completeness (%)
0                                                Year        100.000000
1                                              Entity        100.000000
2                                                Code        100.000000
3                                      GDP per Capita         96.153846
4                                          Population         96.153846
5   Government expenditure on education, total (% ...         98.076923
6   Government expenditure on primary education as...         30.769231
7                                Annual Working Hours         96.153846
8                                Consumer Price Index         67.307692
9                              Government Expenditure         55.769231
10                          Foreign Direct Investment         78.846154
11                                  Unemployment rate         57.692308
12                      Government health expenditure         36

## KPIs of Raw Data

In [11]:
consistency_df, average_consistency = calculate_consistency(rawdata, testdata)
reliability_df, average_reliability = calculate_reliability(rawdata, testdata)
completeness_df, overall_completeness = calculate_completeness(rawdata)

# Merge the results into a single DataFrame
combined_kpis_df = consistency_df.merge(reliability_df, on='Column', how='outer').merge(completeness_df, on='Column', how='outer')

# Optionally, fill missing values if any exist after merging
combined_kpis_df.fillna('N/A', inplace=True)

# Print the combined KPIs DataFrame
print(f"Average Consistency: {average_consistency:.2f}%")
print(f"Average Reliability: {average_reliability:.2f}%")
print(f"Overall Completeness: {overall_completeness:.2f}%")
combined_kpis_df

Average Consistency: 58.74%
Average Reliability: 78.45%
Overall Completeness: 74.39%


,Column,Consistency,Reliability,Completeness (%)
0,Year,100.00%,100.00%,100.000000
1,Entity,100.00%,100.00%,100.000000
2,Code,100.00%,100.00%,100.000000
3,GDP per Capita,0.00%,-331.51%,96.153846
4,Population,0.00%,99.36%,96.153846
5,"Government expenditure on education, total (% ...",1.92%,97.49%,98.076923
6,Government expenditure on primary education as...,100.00%,100.00%,30.769231
7,Annual Working Hours,100.00%,100.00%,96.153846
8,Consumer Price Index,30.77%,97.82%,67.307692
9,Government Expenditure,100.00%,100.00%,55.769231


# Develop Data Quality
Choose most important problem data `GDP per Capita` and `Population` to improve

In [12]:
newdata = rawdata

columns_to_replace = ['GDP per Capita', 'Population']
for column in columns_to_replace:
    if column in newdata.columns and column in testdata.columns:
        newdata[column] = testdata[column]
newdata.to_csv('New_data.csv', index=False)
newdata

,Year,Entity,Code,GDP per Capita,Population,"Government expenditure on education, total (% of GDP)",Government expenditure on primary education as % of GDP (%),Annual Working Hours,Consumer Price Index,Government Expenditure,...,Government health expenditure,Life Expectancy,Government revenues,Age dependency ratio,Research and development spending,Taxes on goods and services (as a share of GDP),"Taxes on income, profits and capital gains (TIPCG) (as a share of GDP)",Human Development Index,Gender Development Index,Productivity
0,1970,China,CHN,113.162578,818315000,NaN,NaN,1976.3120,NaN,NaN,...,NaN,NaN,NaN,80.132945,NaN,NaN,NaN,NaN,NaN,1. 4994909
1,1971,China,CHN,118.654144,841105000,1.504148,NaN,1975.7937,NaN,NaN,...,NaN,57.472110,NaN,79.802335,NaN,NaN,NaN,NaN,NaN,1.5798508
2,1972,China,CHN,131.885559,862030000,1.620546,NaN,1975.5077,NaN,NaN,...,NaN,58.388600,NaN,78.953924,NaN,NaN,NaN,NaN,NaN,1. 492279
3,1973,China,CHN,157.089111,881940000,1.697113,NaN,1975.3647,NaN,NaN,...,NaN,59.367887,NaN,78.661213,NaN,NaN,NaN,NaN,NaN,1. 5905421
4,1974,China,CHN,160.147688,900350000,1.754547,NaN,1975.1897,NaN,NaN,...,NaN,60.144828,NaN,78.936067,NaN,NaN,NaN,NaN,NaN,1. 5722494
5,1975,China,CHN,178.339614,916395000,1.740779,NaN,1974.8975,NaN,NaN,...,NaN,60.702793,NaN,79.252391,NaN,NaN,NaN,NaN,NaN,1. 5965048
6,1976,China,CHN,165.404262,930685000,1.779506,NaN,1974.2075,NaN,NaN,...,NaN,61.613119,NaN,79.236954,NaN,NaN,NaN,NaN,NaN,1. 4927917
7,1977,China,CHN,185.420537,943455000,1.807332,NaN,1973.4353,NaN,NaN,...,NaN,62.365282,NaN,77.057473,NaN,NaN,NaN,NaN,NaN,1. 5616083
8,1978,China,CHN,228.519313,956165000,2.022968,NaN,1972.7274,NaN,NaN,...,NaN,62.818361,NaN,73.129774,NaN,NaN,NaN,NaN,NaN,1. 7562629
9,1979,China,CHN,272.146923,969005000,2.015320,NaN,1972.1038,NaN,NaN,...,NaN,63.483663,NaN,70.219607,NaN,NaN,NaN,NaN,NaN,1. 7937474


## Re-compute KPIs

In [13]:
consistency_df, average_consistency = calculate_consistency(newdata, testdata)
reliability_df, average_reliability = calculate_reliability(newdata, testdata)
completeness_df, overall_completeness = calculate_completeness(newdata)

# Merge the results into a single DataFrame
combined_kpis_df = consistency_df.merge(reliability_df, on='Column', how='outer').merge(completeness_df, on='Column', how='outer')

# Optionally, fill missing values if any exist after merging
combined_kpis_df.fillna('N/A', inplace=True)

# Print the combined KPIs DataFrame
print(f"Average Consistency: {average_consistency:.2f}%")
print(f"Average Reliability: {average_reliability:.2f}%")
print(f"Overall Completeness: {overall_completeness:.2f}%")
combined_kpis_df

Average Consistency: 67.83%
Average Reliability: 98.09%
Overall Completeness: 74.74%


,Column,Consistency,Reliability,Completeness (%)
0,Year,100.00%,100.00%,100.000000
1,Entity,100.00%,100.00%,100.000000
2,Code,100.00%,100.00%,100.000000
3,GDP per Capita,100.00%,100.00%,100.000000
4,Population,100.00%,100.00%,100.000000
5,"Government expenditure on education, total (% ...",1.92%,97.49%,98.076923
6,Government expenditure on primary education as...,100.00%,100.00%,30.769231
7,Annual Working Hours,100.00%,100.00%,96.153846
8,Consumer Price Index,30.77%,97.82%,67.307692
9,Government Expenditure,100.00%,100.00%,55.769231


**The indicators for measuring data quality have all increased**